In [2]:
# Cell 1: Import Libraries and Load Data
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set(style="whitegrid")

# File paths
csv_file_path = "MachineLearningRating_v3.csv"

# Load data
from scripts.helpers import load_data,create_groups, perform_chi2_test, perform_t_test




ModuleNotFoundError: No module named 'scripts.helpers'